In [0]:
'''
Iniciando build_country_dim
'''

'\nIniciando build_country_dim\n'

Importando funções comuns para uso no notebook


In [0]:
%run
./shared_gold_functions

Inicializando sessão spark, definindo o schema da tabela final e importando dados da tabela silver

In [0]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.types import *
from pyspark.sql import functions as F
import urllib

# Inicializa a sessão Spark
spark = SparkSession.builder.appName("accidents_analysis").getOrCreate()

schema = StructType([
   StructField("country_code", StringType(), False,metadata={'comment':"Unique ID, PK"}),
   StructField("country_name", StringType(), False)])

countries_silver_df = spark.read.format("delta").load('dbfs:/user/hive/warehouse/silver_database.db/silver_countries')

countries_silver_df.display()


country_name,country_code
Sudan,ST
Sweden,SE
Afghanistan,YA
Albania,ZA
China,B
Georgia,4L
Iraq,YI
Caribbean Netherlands,PJbes
Cape Verde,D4
Greenland,OYg


## Processamento

Na etapa de processamento, vamos dropar eventuais duplicatas e criar novo df com o schema

In [0]:
countries_silver_df = countries_silver_df.dropDuplicates()

data = countries_silver_df.select("country_code", "country_name").collect()

countries_dim_df = spark.createDataFrame(data,schema)

countries_dim_df.display()

country_code,country_name
ST,Sudan
SE,Sweden
YA,Afghanistan
ZA,Albania
B,China
4L,Georgia
YI,Iraq
PJbes,Caribbean Netherlands
D4,Cape Verde
OYg,Greenland


##Avaliando a qualidade dos dados
- Testar se não há nulos em country_code
- Testar se não há nulos em country_name

In [0]:
# Testando a qualidade dos dados
test_col_not_null(countries_dim_df,'country_code')
test_col_not_null(countries_dim_df,'country_name')

Avaliando a condição country_code não contem nulos
Avaliando a condição country_name não contem nulos


## Registrando dados no banco
Após aprovação, salvamos o dado sanitizado no banco e verificamos se tivemos sucesso

In [0]:
# Cria o banco de dados se ele não existir
database_name = 'gold_database'
table_name = 'gold_countries_dim'


spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Salva o DataFrame como tabela Delta
countries_dim_df.write.mode("overwrite").format("delta").option("mergeSchema", "true").saveAsTable(
    f"{database_name}.{table_name}"
)

In [0]:
%sql
SELECT * from gold_database.gold_countries_dim

country_code,country_name
T2,Tuvalu
YK,Syria
EI,Ireland
F-Oma,Martinique
UK,Uzbekistan
VP-L,British Virgin Islands
TL,Central African Republic
T7,San Marino
OM,Slovakia
TG,Guatemala


In [0]:
'''
Finalizando build_country_dim
'''

'\nFinalizando build_country_dim\n'